We start by installing the Google Developers Python client:

`sudo pip install --upgrade google-api-python-client`

(https://developers.google.com/api-client-library/python/start/installation)

In [15]:
from oauth2client.client import flow_from_clientsecrets

flow = flow_from_clientsecrets('picasa/client_secret.json',
                               scope='https://picasaweb.google.com/data/',
                               redirect_uri='http://ipython-panos.ipeirotis.com:5000/oauth2callback')

auth_uri = flow.step1_get_authorize_url()
print auth_uri

https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fpicasaweb.google.com%2Fdata%2F&redirect_uri=http%3A%2F%2Fipython-panos.ipeirotis.com%3A5000%2Foauth2callback&response_type=code&client_id=61623328740-c1619c0d1vemj9kagupo6r7d154et8vs.apps.googleusercontent.com&access_type=offline


In [16]:
# In this cell, we just configure the server. We will start it, 
# using the command webserver.run(host='0.0.0.0') in the next cell

# Flask is a webserver
from flask import Flask, request

# We will use that part to issue a request to Jawbone
import requests
import json

# To get the server to run in the background
import threading

# This part is required to come up with the pop-up with which Google will ask for permissions
from IPython.display import display
from IPython.display import Javascript as JS

# This is used to store the credentials to the filesystem
from oauth2client.file import Storage
storage = Storage('picasa/credentials.json')

# Initialize the embedded web server
webserver = Flask("PicasaOAuth")

# The function below will implement the sole functionality of our server.
# The Jawbone API will first authenticate the user and then direct the browser
# to http://<your ip>:5000/receiveCode in order to send the code to the app.
# 
# This is the place where the webserver will receive the call from Jawbone
# with the authentication code (Step 3 in https://jawbone.com/up/developer/authentication)
# Notice that we will use the http://<your ip>:5000/receiveCode address below as the redirect URI 
@webserver.route("/oauth2callback")
def oauth_helper():
    code = request.args.get('code')
    credentials = flow.step2_exchange(code)
    storage.put(credentials)


    
    
    # It is safe (and convenient) to shut down the web server after this request
    stop_server()
    
    # What we return here has no real impact on the functionality of the code
    return '<html><body>Code: <b>'+code+'</b></body></html>'


def start_server():
    webserver.run(host='0.0.0.0', port=5000)
    return
    
def stop_server():
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return
    

In [17]:
import threading

# The code below is necessary to pop up the login window from Jawbone
# The login still start in a separate thread, in order to allow the execution
# to continue, and run the "start_server()" call below.
threading.Timer(1, lambda: display(JS("window.open('%s')" % auth_uri))).start()

# And now start the webserver so that we can receive the answer from Google API
start_server()

<IPython.core.display.Javascript object>

In [18]:
import httplib2
from apiclient.discovery import build

# Read the credentials from the file
credentials = storage.get()        
http = httplib2.Http()
http = credentials.authorize(http)
service = build('calendar', 'v3', http=http)

NameError: name 'code' is not defined